In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import scipy
from scipy import misc
from PIL import Image
from keras import regularizers
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
labels=[]

count=0
for dirname,_,filenames in os.walk('/kaggle/input/plant-seedlings-classification/train'):
    d=dirname.split("/")
    for file in filenames:
        count+=1
    if(len(d)==5):
        continue
    labels.append(d[5])
labels=np.array(labels)
print(labels , count)

    

In [ ]:
x_train=np.zeros((4750,256,256,3))
y_train=np.zeros(4750)
index=0

def resize_images(img):

  img = np.array(img).astype(np.uint8)
  #print(img.dtype)
  res = cv2.resize(img,(256,256), interpolation = cv2.INTER_CUBIC)
  return res
for dirname,_,filenames in os.walk('/kaggle/input/plant-seedlings-classification/train'):
    d=dirname.split("/")
    for file in filenames:
        arr=np.array(Image.open(os.path.join(dirname,file)))
        arr=resize_images(arr)
        x_train[index]=arr[:,:,:3]
        result=np.where(labels == d[5])
        y_train[index]=result[0][0]
        
        index+=1
x_train=x_train.astype("float32")/255.0
print(x_train[0])
    
        

In [ ]:
print(x_train[0,:,:,:])

In [ ]:
def my_model():
  inputs=keras.Input(shape=(256,256,3))

  x=layers.Conv2D(32,3,padding='same',kernel_regularizer=regularizers.l2(0.01))(inputs)
  x=keras.activations.relu(x)
  
  x=layers.MaxPooling2D((2,2))(x)
  x=layers.Conv2D(32,3,padding='same',kernel_regularizer=regularizers.l2(0.01))(inputs)
  x=keras.activations.relu(x)
  x=layers.MaxPooling2D((2,2))(x)  
  x=layers.Conv2D(64,3,padding='same',kernel_regularizer=regularizers.l2(0.01))(x)
  x=keras.activations.relu(x)
  x=layers.MaxPooling2D((3,3))(x)
  x=layers.Conv2D(64,3,padding='same',kernel_regularizer=regularizers.l2(0.01))(x)
  x=keras.activations.relu(x)
 
  
  x=layers.Flatten()(x)
  x=layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.01))(x)
  outputs=layers.Dense(12,activation='softmax')(x)
  x=layers.Dropout(0.5)(x)
  model=keras.Model(inputs=inputs,outputs=outputs)
  return model
model=my_model()
print(model.summary())

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer="adam",
    metrics=["accuracy"],
      
)


In [ ]:
model.fit(x_train,y_train,batch_size=64,epochs=2)

In [ ]:
indexs=[]
for dirname,_,filenames in os.walk('/kaggle/input/plant-seedlings-classification/test'):
    d=dirname.split("/")
    indexs=filenames
        

In [ ]:
x_test=np.zeros((794,256,256,3))
index=0


for dirname,_,filenames in os.walk('/kaggle/input/plant-seedlings-classification/test'):
    d=dirname.split("/")
    for file in filenames:
        arr=np.array(Image.open(os.path.join(dirname,file)))
        arr=resize_images(arr)
        x_test[index]=arr[:,:,:3]
        index+=1
        
x_test=x_test.astype("float32")/255.0

In [ ]:
prediction = model.predict(x_test)

In [ ]:
print(prediction[0])
pred_lab=labels[np.argmax(prediction,axis=1)]

In [ ]:
output = pd.DataFrame({'file': indexs,
                       'species': pred_lab})
output.to_csv('submission.csv', index=False)